# Discrete gradient operators

This notebook collects my progress towards the definition of the operator

$$\nabla_h:W_h \rightarrow \Theta_h$$

as defined in (Bartels, 2015) §8.2.1. We proceed as follows:

1. Computation of (global) gradient for CG1 functions, i.e. as an operator from CG1 into CG0.
2. Computation of (global) gradient for DKT functions, i.e. as an operator from DKT into CG2^2.
3. Implementation in the assembly process. (*to do*)

In [ ]:
## The usual boilerplate...

from itertools import chain
from dolfin import *
from petsc4py import PETSc
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

np.set_printoptions(precision=3, linewidth=150)

def __nbinit__():
    global __all__
    __all__ = ['compute_discrete_gradient_dense', 
               'compute_discrete_gradient',
               'compute_dkt_gradient']
    global parameters
    # instruct FFC to produce the code for evaluate_basis_derivatives()
    parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

__nbinit__()

# Gradient of a piecewise linear function

** To do:** A rewrite in C++?

Let $u \in V$, where $V$ is a CG1 finite element space. Then $u'$ is piecewise constant (i.e. it is in DG0). We want to compute the matrix $D_h$ for 

$$\nabla_h:V \rightarrow V'$$

such that the $j$-th coefficient of $u'$ in $V'$, $(\nabla u)_j = (\nabla_h u)_j =: u_j'$. Note that we are using the same  notation for a function $u$ and for the vector of its coefficients in the basis of the space it lives on.

Since $u'$ is not in the same space as $u$, in FEniCS we need to `project()` it onto $V$, which is costly. What we are doing is to manually compute the derivative. This should be easy since

$$ u = \sum_i u_i \phi_i $$

where $V = \text{span} \{\phi_i\} $, and therefore, by linearity:

$$ u' = \sum_i u_i \phi_i'. $$

We could use `evaluate_basis_derivatives()` to evaluate the $\phi_i'$ and then we would be able to evaluate $u'$. However, we are interested in obtaining new coefficients $u_i'$ such that

$$ u' = \sum_i u_i \psi_i', $$

where

assembling the transformation matrix by hand.

We define $V' = \text{span} \{\phi_i'\}$, a DG0 space, and compute the linear mapping of dofs from $V$ to $V'$, then apply this mapping to the coefficients of $u$ and construct $u' \in V'$ with these coefficients.

For simplicity, consider first a 1D mesh. Let $i \in \{0,1,...,M\}$ be an index over the cells of the mesh such that:

* $u_i,u_{i+1}$ are the coefficients for the dofs in $V$ over cell $i$
* $u_i'$ is the coefficient for the only dof in $V'$ (the constant 1) over cell $i$

Then we only need to compute $\phi_i'$ for each $i$ to obtain the new vector of coefficients $(u_i')_{i=0}^{N_{V'}}$:

$$
\left(\begin{array}{ccccc}
  \phi_0' & \phi_1' &  &  & \\
  & \phi_1' & \phi_2' &  & \\
  &  & \phi_2' & \phi_3' & \\
  &  &  & \phi_3' & \phi_4'
\end{array}\right)  \left(\begin{array}{c}
  u_0\\
  u_1\\
  u_2\\
  u_3\\
  u_4
\end{array}\right) = \left(\begin{array}{c}
  u_0'\\
  u_1'\\
  u_2'\\
  u_3'
\end{array}\right).$$

Note that in this particular instance and with the assumptions made, we obtain a band matrix and computing $u'$ is actually a matter of computing the dot product of a "subset" of the vector of constants $(\phi_0', ..., \phi_{N_{V'}}')$ with $u$.

However in other situations, e.g. in higher dimensions, the ordering of the nodes won't be as convenient. We proceed then more generally as follows: Let $i$ run over all cell indices and let $\iota_i$ be the local-to-global mappings for $V$ and $\iota_i'$ for $V'$. Initialise the matrix $M$ of the discrete gradient to zero. At each step of the construction of $M$ we edit row $r = \iota_i'(0)$, i.e. the row whose product by $(u)_j$ will return the coefficient for the dof in $V'$ corresponding to cell $i$.

Now, for every vertex $v_j,\ j \in \{0,...,d\}$ in cell $i$ compute the value of the derivative of the global dof $\phi_{\iota_r(j)}$ and set

$$M_{r j} = \phi_{\iota_r(j)},\ j \in \{0,...,d\}.$$

In [ ]:
def discrete_gradient_operator_dense(V, Vp):
    """ Build dense matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type CG1.
        Vp: Target (Vector)FunctionSpace of type DG0.
        
    Returns
    -------
        Dh: np.ndarray of shape (Vp.dim(), V.dim()) such that,
            for u in V:
        
                np.dot(Dh, u.vector().array())
              
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'CG1', "Need a CG1 source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x DG0' % gdim or\
           Vp.ufl_element().shortstr()[:3] == 'DG0',\
           "I need a DG0 (Vector)FunctionSpace of the right number of components"
    assert Vp.mesh().num_cells() * gdim == Vp.dim(),\
           "FunctionSpace dimensions don't match."
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Meshes don't match"

    dm = V.dofmap()
    dmp = Vp.dofmap()
    e = V.element()
    coords = V.tabulate_dof_coordinates().reshape((-1, gdim))
    coordsp = Vp.tabulate_dof_coordinates().reshape((-1, gdim))

    Dh = np.zeros((Vp.dim(), V.dim()))
    warning("Assuming all cells have the same number of dofs")
    vals = np.zeros(dm.num_element_dofs(0)*gdim)
    for cell_id in range(Vp.mesh().num_cells()):
        rows = dmp.cell_dofs(cell_id)
        columns = dm.cell_dofs(cell_id)
        dof_coords = coords[columns,:]
        point = dof_coords[0] # Any point in the (closure of the) cell will do.
        e.evaluate_basis_derivatives_all(1, vals, point, dof_coords, 1)
        #print("rows=%s, columns=%s, point=%s, dof_coords=%s, vals=%s" %
        #     (rows, columns, point, dof_coords, vals.round(2)))
        # Reshape [df_1/dx, df_1/dy, df_2/dx, df_2/dy, ...] into
        # [[df_1/dx, df_2/dx, ...],[df_1/dy, df_2/dy, ...]] :
        #Dh[FIXTHIS:"rows, columns"] = vals.reshape((-1, gdim)).T.copy()
        for offset in range(gdim):
            Dh[rows[offset], columns] = vals[offset::gdim].copy()

    return Dh

def compute_discrete_gradient_dense(V, Vp, u):
    """ Returns the discrete gradient of u.
    NOTE: this implementation uses dense matrices internally.

    Arguments
    ---------
        V: Source FunctionSpace of type CG1. (redundant...)
        Vp: Target (Vector)FunctionSpace of type DG0.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """

    Dh = discrete_gradient_operator_dense(V, Vp)
    up = Function(Vp)
    up.vector().set_local(np.dot(Dh, u.vector().array()))
    up.vector().apply('insert')

    return up

In order to test the previous code we start with an expression whose derivative we can compute:

In [ ]:
LEFT, RIGHT = -pi, 2*pi
mesh = IntervalMesh(100, LEFT, RIGHT)
V = FunctionSpace(mesh, "CG", 1)
Vp = FunctionSpace(mesh, "DG", 0)

#x = SpatialCoordinate(mesh)
#f = sin(x[0])
#u = project(f, V)
u = interpolate(Expression("sin(x[0])", element=V.ufl_element()), V)
dg_u = compute_discrete_gradient_dense(V, Vp, u)

# Test
#up = project(f.dx(0), Vp)
up = interpolate(Expression("cos(x[0])", element=Vp.ufl_element()), Vp)
print("Error: %e" % (dg_u.vector() - up.vector()).norm('linf'))
plot(up)
plot(dg_u)
_ = pl.xlim((LEFT*1.1, RIGHT*1.1)), pl.ylim((-1.1,1.1))

## A detour using projections

The following method computes the solution via a projection onto DG0 but instead of letting FEniCS assemble the mass matrix, we make use of the fact that it is diagonal with entries given by the cell volume. Then it is trivial to invert it and multiply the rhs of the projection problem. The idea and code are taken from [this question on FEniCS Q&A](https://fenicsproject.org/qa/12634/gradient-of-a-cg-order-one-element?show=12646#a12646).

In [ ]:
mesh = UnitCubeMesh(10, 10, 10)
W = FunctionSpace(mesh, 'CG', 1)
Q = VectorFunctionSpace(mesh, 'DG', 0)
p, q = TrialFunction(Q), TestFunction(Q)

f = Function(W)
f_vec = f.vector()
f_vec.set_local(np.random.rand(f_vec.local_size()))
f_vec.apply('insert')

We first compute the gradient via a projection. **NOTE** that the computation for the form $L$ will have to interpolate $f$, which is in "W", into the space $T$ where the test function $q$ lives in order to compute the integral. [WHY?]

In [ ]:
%%time

L = inner(grad(f), q)*dx
M = assemble(inner(p, q)*dx)
b = assemble(L)

grad_f0 = Function(Q)
x0 = grad_f0.vector()
_ = solve(M, x0, b)

MiroK's idea (which amounts to building our matrix $D_h$) in the forum is to "assemble the action of Minv onto the rhs - no mass matrix needed". The improvement in speed is huge:

In [ ]:
%%time

MinvL = (1/CellVolume(Q.mesh()))*inner(grad(f), q)*dx
x = assemble(MinvL)

In [ ]:
grad_f = Function(Q, x)
print("Error: %e" % (x - x0).norm('linf'))

Note that in the first approach we compute $M$, then solve $M x = b$ and in the second we compute $M^{-1}$ directly to obtain $x = M^{-1} b$.

### How does the assembly of the projection problem work?

Write the compiled form to a file and check `dgrad_cell_integral_0_otherwise::tabulate_tensor()` for the implementation. Recall that `u.dx()` in the form is the derivative of a terminal node in UFL so it is the responsibility of FFC to fill that. How does this work again?

In [ ]:
import ffc
with open("/tmp/dgrad.h", "wt") as fd:
    out = ffc.compile_form(MinvL, prefix="dgrad")
    fd.write(out[0])

## And back

We can use our function instead. This will be much slower and also use a lot of memory because of the dense matrix we are using. The error is different, though. **Could this be significant?**

In [ ]:
%%time

dg_f = compute_discrete_gradient_dense(W, Q, f)

In [ ]:
print("Error: %e" % (dg_f.vector() - grad_f0.vector()).norm('linf'))

## Using the PETSc backend

By using PETSc We can achieve a (back-of-the-napkin) speed improvement of up to 50 times wrt. the implementation with dense matrices and stay in the same range of running times than `project()`. We can only test for manageable matrix sizes but we can now of course handle bigger matrices. The speedup depends greatly on the size of the matrices.

In [ ]:
def discrete_gradient_operator(V, Vp):
    """ Build sparse PETSc matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type CG1.
        Vp: Target (Vector)FunctionSpace of type DG0.
        
    Returns
    -------
        Dh: PETSc sparse matrix such that for u in V,
              Dh * u.vector().array()
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """   
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'CG1',\
           "Need a CG1 source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x DG0' % gdim or\
           Vp.ufl_element().shortstr()[:3] == 'DG0',\
           "I need a DG0 (Vector)FunctionSpace of the right number of components"
    assert Vp.mesh().num_cells() * gdim == Vp.dim(),\
           "FunctionSpace dimensions don't match"
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Number of cells in the meshes don't match"

    dm = V.dofmap()
    dmp = Vp.dofmap()
    e = V.element()
    coords = V.tabulate_dof_coordinates().reshape((-1, gdim))
    coordsp = Vp.tabulate_dof_coordinates().reshape((-1, gdim))

    Dh = PETSc.Mat()
    Dh.create(PETSc.COMM_WORLD)  # What is this?
    Dh.setSizes([Vp.dim(), V.dim()])
    Dh.setType("aij")
    Dh.setUp()
    warning("Assuming all cells have the same number of dofs")
    vals = np.zeros(dm.num_element_dofs(0)*gdim)
    # TODO: check this for parallel operation...
    istart, iend = Dh.getOwnershipRange()
    print ("This process owns the range %d - %d" % (istart, iend))
    for cell_id in range(Vp.mesh().num_cells()):
        rows = dmp.cell_dofs(cell_id)
        rows = rows[(rows>=istart) & (rows < iend)]
        columns = dm.cell_dofs(cell_id)
        dof_coords = coords[columns,:]
        point = dof_coords[0] # Any point in the (closure of the) cell will do.
        e.evaluate_basis_derivatives_all(1, vals, point, dof_coords, 1)
        #print("rows=%s, columns=%s, point=%s, dof_coords=%s, vals=%s" %
        #     (rows, columns, point, dof_coords, vals.round(2)))
        # Reshape [df_1/dx, df_1/dy, df_2/dx, df_2/dy, ...] into
        # [[df_1/dx, df_2/dx, ...],[df_1/dy, df_2/dy, ...]] :
        # NOTE: the copy() is necessary!
        Dh.setValues(rows, columns, vals.reshape((-1, gdim)).T.copy())
    Dh.assemble()
    return Dh

def compute_discrete_gradient(V, Vp, u):
    """ Returns the discrete gradient of u.

    Arguments
    ---------
        V: Source FunctionSpace of type CG1. (redundant...)
        Vp: Target (Vector)FunctionSpace of type DG0.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """
    Dh = discrete_gradient_operator(V, Vp)
    du = Function(Vp)
    petsc_u = as_backend_type(u.vector()).vec()
    petsc_du = as_backend_type(du.vector()).vec()
    
    Dh.mult(petsc_u, petsc_du)
    du.vector().apply('insert')

    return du

In [ ]:
# instruct FFC to produce the code for evaluate_basis_derivatives()
parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

LEFT, RIGHT = -pi, 2*pi
mesh = IntervalMesh(1000, LEFT, RIGHT)
V = FunctionSpace(mesh, "CG", 1)
Vp = FunctionSpace(mesh, "DG", 0)

#Dh = discrete_gradient_operator(V, Vp)
u = interpolate(Expression("sin(x[0])", degree=1), V)
du0 = interpolate(Expression("cos(x[0])", degree=0), Vp)

In [ ]:
%%time
du = compute_discrete_gradient(V, Vp, u)

In [ ]:
print("Error: %e" % (du.vector() - du0.vector()).norm('linf'))
_ = plot(du)

This is faster than the solution with dense matrices but obviously still much slower (a little below 40 times) than building $M^{-1}$ directly because of the hideously slow python loops:

In [ ]:
%%time

dgrad_f = compute_discrete_gradient(W, Q, f)

In [ ]:
print("Error: %e" % (dgrad_f.vector() - grad_f0.vector()).norm('linf'))

# Discrete gradient for DKT

Implementation of

$$\nabla_h:W_h \rightarrow \Theta_h$$

for Functions over the whole space. Recall that the operator $\nabla_h$ is given cell-wise by the matrix

$$M = .$$

Since most of it won't change, we compute it in two steps: an initialization step, which sets the fixed entries and a per-cell step, where we set the entries for each cell. **Note:** because this is just a hack, we use temporaries, hence the need to encapsulate everything in a class.

In [ ]:
class DKTCellGradient(object):
    """ Computes the transformation matrix for $\nabla_h$ over one cell.
    
    This assumes that the dofs of the "source" cell (in DKT) are ordered as
    
         w1, w1_x, w1_y, w2, w2_x, w2_y, w3, w3_x, w3_y,
         
    i.e. point evaluation, partial derivatives for each vertex in turn, and
    those of the "target" cell (in P_2^2) as
    
        t1_0, t1_1, t2_0, t2_1, t3_0, t3_1, ts1_0, ts1_1, ts2_0, ts2_1, ts3_0, ts3_1
        
    i.e. values at the three vertices, then values at the the midpoints of the
    opposite sides.
    """
    def __init__(self, m:int, n:int):
        """ Initialize the matrix with m rows (target's local space_dimension())
        and n columns (source's local space_dimension()."""
        self.M = np.zeros((m, n), dtype=np.float)
        rows = [[0,1], [2,3], [4,5], [6,7], [8,9], [10,11]]
        # Hack to make fancy indexing with lists work:
        # if one of rows[i], self.cols[j] has shape 2,1, broadcasting will actually
        # compute the outer product of rows[i] and cols[j], instead of a "zip"
        # of sorts.
        self.rows = list(map(lambda x: np.array(x).reshape(2,1), rows))
        self.cols = [0, [1,2], 3, [4,5], 6, [7,8]]
        #self.cols = list(map(lambda x: np.array(x).reshape((-1,1)), self.cols))
        # TODO: Note that storing these copies of the identity is silly
        # When we apply the operator, it would make more sense to just copy
        # the partial derivative dofs (which is what these first 3 multiplications 
        # with the identity do), then multiply with the lower part of M...
        # (But we still need these here if we insist on building a global matrix
        # which acts on all dofs of the space simultaneously)
        self.M[self.rows[0], self.cols[1]] = np.eye(2)
        self.M[self.rows[1], self.cols[3]] = np.eye(2)
        self.M[self.rows[2], self.cols[5]] = np.eye(2)

        # We are only defined for triangles (to do: assert this)
        # tdim = 2, num_vertices = 3

        # Init temporaries
        self.tt = np.empty((3, 2))
        self.TT = np.empty((3, 2, 2))
        self.ss = np.empty(3)

    def update(self, cell:Cell):
        """Update the mutating entries of M for the given cell."""
        cc = cell.get_vertex_coordinates().reshape(-1, 2)
        self.tt[0] = (cc[2] - cc[1])
        self.tt[1] = -(cc[0] - cc[2])  # Magic: why was this minus sign here again?
        self.tt[2] = (cc[1] - cc[0])
        for i in range(3):
            self.ss[i] = np.linalg.norm(self.tt[i], ord=2)
            self.tt[i] /= self.ss[i]
            self.TT[i] = -3./4 * np.outer(self.tt[i], self.tt[i])
            # HERE! if we don't use the hierarchical basis we need to add this, see...
            self.TT[i] += 0.5*np.eye(2)
            self.tt[i] *= -3./(2*self.ss[i])

        self.M[self.rows[3], self.cols[2]] = self.tt[0].copy().reshape(2,1)
        self.M[self.rows[3], self.cols[3]] = self.TT[0].copy()
        self.M[self.rows[3], self.cols[4]] = -self.tt[0].copy().reshape(2,1)
        self.M[self.rows[3], self.cols[5]] = self.TT[0].copy()

        self.M[self.rows[4], self.cols[0]] = self.tt[1].copy().reshape(2,1)
        self.M[self.rows[4], self.cols[1]] = self.TT[1].copy()
        self.M[self.rows[4], self.cols[4]] = -self.tt[1].copy().reshape(2,1)
        self.M[self.rows[4], self.cols[5]] = self.TT[1].copy()

        self.M[self.rows[5], self.cols[0]] = self.tt[2].copy().reshape(2,1)
        self.M[self.rows[5], self.cols[1]] = self.TT[2].copy()
        self.M[self.rows[5], self.cols[2]] = -self.tt[2].copy().reshape(2,1)
        self.M[self.rows[5], self.cols[3]] = self.TT[2].copy()

We now aggregate the previous cell-wise operator into one acting over discrete functions over the whole space.

**Fixme:** this is very inefficient:

* There is no need to create and multiply by the first 6 rows of the cell matrix. If we didn't create a "global matrix", i.e. acting on whole vectors of coefficients, but instead proceeded cell by cell this would be faster (albeit at the cost of a python loop, which might prove to be even worse)
* If I keep the whole matrix, I should at least try to preinit the sparsity pattern.
* It should be possible to loop through all facets instead of cells. This should reduce computing time because of the fewer updates to the matrix

In [ ]:
def dkt_gradient_operator(V, Vp):
    """ Build sparse PETSc matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type DKT.
        Vp: Target (Vector)FunctionSpace of type P2
            (WITHOUT a hierarchical basis)
        
    Returns
    -------
        Dh: PETSc sparse matrix such that for u in V,
              Dh * u.vector().array()
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """   
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'DKT',\
           "I need a DKT source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x CG2' % gdim,\
           "I need a CG2 VectorFunctionSpace with %d components" % gdim
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Number of cells in the meshes don't match"
    dm = V.dofmap()
    dmp1, dmp2 = Vp.sub(0).dofmap(), Vp.sub(1).dofmap()
    e = V.element()
    ep = Vp.element()
    tdim = e.topological_dimension()
    assert tdim == 2, "I need topological dimension 2 (was %d)" % tdim
    coords = V.tabulate_dof_coordinates().reshape((-1, tdim))

    Dh = PETSc.Mat()
    Dh.create(PETSc.COMM_WORLD)
    Dh.setSizes([Vp.dim(), V.dim()])
    Dh.setType("aij")
    Dh.setUp()
    # In order not to assume the following it would be enough
    # to create the array inside the loop...
    warning("Assuming all cells have the same number of dofs")

    ## Preinit element-wise gradient
    grad = DKTCellGradient(ep.space_dimension(), e.space_dimension())

    # TODO: check this for parallel operation...
    istart, iend = Dh.getOwnershipRange()
    #print ("This process owns the range %d - %d" % (istart, iend))

    visited = set()
    # Wait, this could be ok
    #print("##########################################")
    #print("#FIXME!!!! dest_rows is completely WRONG!#")
    #print("##########################################")
    
    for cell_id in range(V.mesh().num_cells()):
        cell = Cell(V.mesh(), cell_id)
        facets = cell.entities(2)
        grad.update(cell)
        # Massage dofs into the ordering we require with
        # both coordinates for each vector-valued dof consecutive.
        dest_rows = np.array(list(chain.from_iterable(zip(dmp1.cell_dofs(cell_id),
                                                          dmp2.cell_dofs(cell_id)))),
                             dtype=np.intc)
        dest_rows = dest_rows[(dest_rows>=istart) & (dest_rows < iend)]
        # dofs in V are already in the proper ordering (point eval, gradient eval, ...)
        dest_columns = dm.cell_dofs(cell_id)

        # NOTE: the copy() is necessary!
        #print("Setting:\n\trows=%s\n\tcols=%s\nwith:\n%s" % (dest_rows, dest_columns, M))
        Dh.setValues(dest_rows, dest_columns, grad.M.copy())
        assert len(dest_rows)*len(dest_columns) ==  grad.M.size, "duh"
    Dh.assemble()
    return Dh

In [ ]:
def compute_dkt_gradient(V, Vp, u):
    """ Returns the discrete gradient of u.

    Arguments
    ---------
        V: Source FunctionSpace of type DKT. (redundant...)
        Vp: Target VectorFunctionSpace of type CG2.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """
    Dh = dkt_gradient_operator(V, Vp)
    du = Function(Vp)
    petsc_u = as_backend_type(u.vector()).vec()
    petsc_du = as_backend_type(du.vector()).vec()
    
    Dh.mult(petsc_u, petsc_du)
    du.vector().apply('insert')

    return du

## Tests

In [ ]:
import nbimporter
from interpolation import interpolate_hermite
import mshr

domain = mshr.Rectangle(Point(1.0, 1.0), Point(2.0, 2.0))
mesh = mshr.generate_mesh(domain, 10)
W = FunctionSpace(mesh, 'DKT', 3)
T = VectorFunctionSpace(mesh, 'Lagrange', 2, dim=2)

In [ ]:
#%%timeit  #(~24ms)

#  Shapes do not match: <Argument ...> and <Grad ...>.??!?
class Fun(Expression):
    def eval(self, value, x):
        value = x[0]*x[1]
    def value_shape(self):
        return (2,)
#f = Fun(degree=3, domain=mesh)
f = Expression("2*x[0]*x[0]*x[1]/sqrt(x[1])", degree=3, domain=mesh)
gf = project(grad(f), T)

In [ ]:
#%%timeit  #(~237ms)
from utils import make_derivatives
import autograd.numpy as np
@make_derivatives
def fun(x,y):
    return 2*x**2*y/np.sqrt(y)

w = interpolate_hermite(fun, W)
dgf = compute_dkt_gradient(W, T, w)
_ = plot(w)

In [ ]:
if not np.allclose(gf.vector().array(), dgf.vector().array(), atol=1e-11):
    dgfa, gfa = dgf.vector().array(), gf.vector().array()
    difs = np.abs(dgfa-gfa)/np.maximum(np.abs(dgfa), np.abs(gfa))
    print("Results differ. Maximum difference = %.3e, mean relative difference = %.3e, std dev = %.3e" 
          % (np.max(np.abs(dgfa-gfa)), difs.mean(), difs.std()))
    pl.figure(figsize=(20,10))
    pl.subplot(1,2,1)
    plot(mesh, alpha=0.3)
    plot(gf)
    pl.title("Projection (%.1e)" % np.max(np.abs(gfa)))
    pl.subplot(1,2,2)
    plot(mesh, alpha=0.3)
    plot(dgf)
    _ = pl.title("Discrete operator (%.1e)" % np.max(np.abs(gfa)))
else:
    print("The gradients agree!!!")

Note how values at the vertices seem to be correct, yet there is still a difference. Indeed the issue seems to be mostly at the facet dofs. First let's extract bith vertex and facet dofs for each subspace.

In [ ]:
from dofs import plot_dofs, extract_dofs_with_mask

vertex_dofs_0 = extract_dofs_with_mask(T, [0,1,2])   # T.sub(0)
vertex_dofs_1 = extract_dofs_with_mask(T, [6,7,8])   # T.sub(1)
facet_dofs_0 = extract_dofs_with_mask(T, [3,4,5])    # T.sub(0)
facet_dofs_1 = extract_dofs_with_mask(T, [9,10,11])  # T.sub(1)

The following routine will do the plotting of the gradients:

In [ ]:
def plot_field_at_dofs(f:Function, dofs0:list, dofs1:list, **kwargs):
    """ Plots a 2d field over a 2d domain.
    Arguments
    ---------
        f: must be defined over a VectorFunctionSpace of dimension 2
        dofs0, dofs1: dofs for the first and second components of f
    """
    assert f.geometric_dimension() == 2, "Need a 2d vector valued function"
    V = f.function_space()
    assert V.element().topological_dimension() == 2, "Need a 2d domain"
    
    all_coords = V.tabulate_dof_coordinates().reshape(-1,2)
    vf = f.vector().array()
    coords = all_coords[dofs0]
    assert np.all(coords == all_coords[dofs1])  # sanity check
    pl.quiver(coords[:,0], coords[:,1], vf[dofs0], vf[dofs1], **kwargs)

Next we compute the list of dofs where the projected gradient and the one computed by the discrete operator disagree:

In [ ]:
def bad_dofs(f:Function, g:Function, dofs0, dofs1, plot=True):
    """
    f,g: Functions over the same VectorFunctionSpace V of dim=2
    dofs0, dofs1 are dofs in V.sub(0), V.sub(1), i.e. they should be of same length and go in pairs"""
    assert f.geometric_dimension() == g.geometric_dimension() == 2, "Need 2d vector valued functions"
    assert f.function_space().element().topological_dimension() == 2, "Need a 2d domain"
    vf = gf.vector().array()
    vg = dgf.vector().array()
    assert len(dofs0) == len(dofs1)
    num_dofs = len(dofs0)
    bad_dofs = []
    for i,j in zip(dofs0, dofs1): # range(num_vertices):
        try:
            dx, dy = vg[i] / vf[i], vg[j] / vf[j]
            if not (near(dx, 0, eps=1e-6) or near(dx, 1, eps=1e-6)) or\
               not (near(dy, 0, eps=1e-6), near(dy, 1, eps=1e-6)):
                #print("Not colinear with factors = {},{}".format(dx, dy))
                bad_dofs.append(i)
        except Exception as e:
            print("{} Skipping {}".format(e, i))
    print("Computed gradients differ at %.2f%% of the dofs." % 
          (100*len(bad_dofs)/num_dofs))
    if plot:
        plot_dofs(T, bad_dofs, s=60, c='red', alpha=0.6)
        plot_field_at_dofs(f, dofs0, dofs1, alpha=0.7, color='orange')
        plot_field_at_dofs(g, dofs0, dofs1, alpha=0.8, color='green')
    else:
        return bad_dofs

And here are two nice plots:

In [ ]:
pl.figure(figsize=(24,12))
pl.subplot(1,2,1)
bad_dofs(gf, dgf, vertex_dofs_0, vertex_dofs_1)
pl.title("Vertex dofs (%.1e)" % np.mean(np.abs(dgf.vector().array())))
pl.subplot(1,2,2)
bad_dofs(gf, dgf, facet_dofs_0, facet_dofs_1)
_ = pl.title("Facet dofs (%.1e)" % np.mean(np.abs(dgf.vector().array())))

So we are basically ok, up to minor numerical errors (which **may build up!**, so I should be careful here...).

In [ ]:
mesh = UnitSquareMesh(1,1)
W = FunctionSpace(mesh, 'DKT', 3)
T = VectorFunctionSpace(mesh, 'Lagrange', 2, dim=2)

w = Function(W)
dmw = W.dofmap()
wdofs_cell0 = dmw.cell_dofs(0)
wdofs_cell1 = dmw.cell_dofs(1)
new_vals = np.zeros_like(w.vector().array())
new_vals[wdofs_cell0] = [2,1,1,2,1,1,2,1,1]
w.vector().set_local(new_vals)


Dh = DKTCellGradient(T.element().space_dimension(), W.element().space_dimension())
Dh.update(Cell(W.mesh(), 0))
product = Dh.M@w.vector().array()[wdofs_cell0]

dw2 = Function(T)
dm0 = T.sub(0).dofmap()
dm1 = T.sub(1).dofmap()
tdofs0_cell0 = dm0.cell_dofs(0)
tdofs1_cell0 = dm1.cell_dofs(0)
#### <strike>DON'T FIXME!!!! this is NOT WRONG:</strike>
dest_rows = np.array(list(chain.from_iterable(zip(tdofs0_cell0,tdofs1_cell0))), dtype=np.intc)
print(tdofs0_cell0, tdofs1_cell0)
print(dest_rows)
#### found manually:
dest_rows = [ 2, 3, 12, 13, 4, 5, 14, 15, 6, 7, 16, 17]
new_vals2 = np.zeros_like(dw2.vector().array())
new_vals2[dest_rows] = product
dw2.vector().set_local(new_vals2)
print(product)
print(Dh.M)

pl.figure(figsize=(12,6))
plot(T.mesh(), alpha=0.3)
vertex_dofs_0 = extract_dofs_with_mask(T, [0,1,2])   # T.sub(0)
vertex_dofs_1 = extract_dofs_with_mask(T, [6,7,8])   # T.sub(1)
facet_dofs_0 = extract_dofs_with_mask(T, [3,4,5])    # T.sub(0)
facet_dofs_1 = extract_dofs_with_mask(T, [9,10,11])  # T.sub(1)
plot_field_at_dofs(dw2, vertex_dofs_0, vertex_dofs_1, color='orange')
plot_field_at_dofs(dw2, facet_dofs_0, facet_dofs_1, color='green')

## Iterating over facets

In [ ]:
from dofs import facets_dofmap

dm = facets_dofmap(T)
dm.facet_dofs[12]